<a href="https://colab.research.google.com/github/liminovna/SpeechandLanguageProcessingV3_Solutions/blob/main/JurafskyNLP_Chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3.1. Write out the equation for trigram probability estimation

$P(w_n|w_{n-2}w_{n-1}) = \frac{C(w_{n-2}w_{n-1}w_n)}{C(w_{n-2}w_{n-1})}$

Now write out all the non-zero trigram probabilities for the *I am Sam* corpus

    <s> I am Sam </s>
    <s> Sam I am </s>
    <s> I do not like green eggs and ham </s>

In [29]:
class TrigramModel():
  def __init__(self, corpus):
    # self.unigrams = {}
    self.bigrams = {}
    self.trigrams = {}

    for s in corpus:
      words = s.lower().split()
      # # appending unigrams
      # for w in words:
      #   if w not in self.unigrams.keys():
      #     self.unigrams[w] = 1
      #   else:
      #     self.unigrams[w] += 1
      # appending bigrams
      cntr = 0
      for i in range(len(words)-1):
        cur_bigram = (words[cntr] + ' ' + words[cntr+1])
        if cur_bigram not in self.bigrams:
          self.bigrams[cur_bigram] = 1
        else:
          self.bigrams[cur_bigram] += 1
        cntr += 1
      # appending trigrams
      cntr = 0
      for i in range(len(words)-2):
        cur_trigram = (words[cntr] + ' ' + words[cntr+1] + ' ' + words[cntr+2])
        if cur_trigram not in self.trigrams:
          self.trigrams[cur_trigram] = 1
        else:
          self.trigrams[cur_trigram] += 1
        cntr += 1

  # def add1smoothing_proba(self, s):
  #   s = s.lower()
  #   words = s.split()
  #   return (self.trigrams[s] + 1) / (self.unigrams[words[0]]+len(self.unigrams))

  def unsmoothed_proba(self, s):
    s = s.lower()
    words = s.split()
    return self.trigrams[s]/self.bigrams[words[0] + ' ' + words[1]]

In [48]:
corpus = [
    '<s> I am Sam </s>',
    '<s> Sam I am </s>',
    '<s> I do not like green eggs and ham </s>',
]

model = TrigramModel(corpus=corpus)

In [49]:
print('Answer: ')
for t in model.trigrams.keys():
  print(f'P({t}) = {model.unsmoothed_proba(t)}')

Answer: 
P(<s> i am) = 0.5
P(i am sam) = 0.5
P(am sam </s>) = 1.0
P(<s> sam i) = 1.0
P(sam i am) = 1.0
P(i am </s>) = 0.5
P(<s> i do) = 0.5
P(i do not) = 1.0
P(do not like) = 1.0
P(not like green) = 1.0
P(like green eggs) = 1.0
P(green eggs and) = 1.0
P(eggs and ham) = 1.0
P(and ham </s>) = 1.0


## 3.2 Calculate the probability of the sentence *i want chinese food*. Give two probabilities -- unsmoothed and an add-one probability. Assume the additional add-1 smoothed probabilities $P(i|<s>)=0.19$ and $P(</s>|food)=0.40$.

In [46]:
sentence = '<s> I want chinese food <\s>'
unsmoothed_proba = 0.25*0.33*0.0065*0.52*0.68
addone_proba = 0.19*0.21*0.0029*0.52*0.4
print('Answer\n',\
      'Unsmoothed probability:', "{:.5f}".format(unsmoothed_proba), '\n',\
      'Add-one probability:', "{:.7f}".format(addone_proba))

Answer
 Unsmoothed probability: 0.00019 
 Add-one probability: 0.0000241


## 3.3 Which of the two probabilities you computed in the previous exercise is higher, unsmoothed or smoothed? Explain why.

Answer:

The unsmoothed probability is higher because we did not give away any bit of it to the events we have not seen, unlike smoothed probability which is what's left after we've 'shaved off' some of the probability mass.

## 3.4 We are given the following corpus:

    '<s> i am sam </s>',
    '<s> sam i am </s>',
    '<s> i am sam </s>',
    '<s> i do not like green eggs and sam </s>',

Using a bigram language model with add-one smoothing, what is P(Sam|am)? Include \<s> and \<\s> in your counts just like any other token.

In [42]:
class BigramModel():
  def __init__(self, corpus):
    self.unigrams = {}
    self.bigrams = {}

    for s in corpus:
      words = s.split()
      # appending unigrams
      for w in words:
        if w not in self.unigrams.keys():
          self.unigrams[w] = 1
        else:
          self.unigrams[w] += 1
      # appending bigrams
      cntr = 0
      for i in range(len(words)-1):
        cur_bigram = (words[cntr] + ' ' + words[cntr+1])
        if cur_bigram not in self.bigrams:
          self.bigrams[cur_bigram] = 1
        else:
          self.bigrams[cur_bigram] += 1
        cntr += 1

  def add1smoothing_proba(self, s):
    s = s.lower()
    words = s.split()
    return (self.bigrams[s] + 1) / (self.unigrams[words[0]]+len(self.unigrams))

  def unsmoothed_proba(self, s):
    s = s.lower()
    words = s.split()
    return self.bigrams[s]/self.unigrams[words[0]]

In [44]:
corpus = [
    '<s> i am sam </s>',
    '<s> sam i am </s>',
    '<s> i am sam </s>',
    '<s> i do not like green eggs and sam </s>',
]

model = BigramModel(corpus=corpus)
print('Answer: ', round(model.add1smoothing_proba('am Sam'), 3))

Answer:  0.214


## 3.5 Suppose we didn't use the end-symbol \<\s>. Train an unsmoothed bigram grammar on the following training corpus without using the end-symbol \<\s>:

    '<s> a b',
    '<s> b b',
    '<s> b a',
    '<s> a a'

In [ ]:
corpus = [
    '<s> a b',
    '<s> b b',
    '<s> b a',
    '<s> a a'
]

model = BigramModel(corpus=corpus)
model.unsmoothed_proba('a a') + model.unsmoothed_proba('a b') + model.unsmoothed_proba('b b') + model.unsmoothed_proba('b a')

1.0

## 3.6 Suppose we train a trigram language model with add-one smoothing on a given corpus. The corpus contains V word types. Express a formula for estimating $P(w3,w1,w2)$, where $w3$ is a word which follows the bigram $(w1,w2)$, in terms of various n-gram counts and V.

$P_{add-one} (w3|w1w2) = \frac{C(w1w2w3)+1}{C(w1w2)+V_2}$

, where $V_2$ is the number of unique bigrams in the corpus

## 3.7 We are given the following corpus, modified from the one in the chapter:
    '<s> i am sam </s>',
    '<s> sam i am </s>',
    '<s> i am sam </s>',
    '<s> i do not like green eggs and sam </s>',

If we use linear interpolation smoothing between a maximum-likelihood bigram
model and a maximum-likelihood unigram model with $\lambda_1 = \frac{1}{2}$ and $\lambda_2 = \frac{1}{2}$ , what is $P(Sam|am)$? Include <s> and </s> in your counts just like any other token.